In [17]:
import pandas as pd
from seml import get_results

In [18]:
df = get_results(
    'dp_timeseries_dp_train_standard_eval_user_level',
    [
        'config.seed',
        'config.estimator_name',
        'config.estimator_kwargs.batch_size',
        'config.dp_optimizer_kwargs.max_grad_norm',
        'config.dp_optimizer_kwargs.noise_multiplier',
        'config.estimator_kwargs.relative_context_length',
        'config.neighboring_relation.size',
        'config.dp_accountant_kwargs.budget_epsilon',
        'result.metrics_test.mean_wQuantileLoss',
        'result.metrics_test.MAPE'
    ],
    to_data_frame=True,
    filter_dict={
        'config.dataset_kwargs.dataset_name': 'traffic',
        'config.estimator_kwargs.trainer_kwargs.max_epochs': 4000,
        'config.top_level_mode': 'sampling_without_replacement',
        'config.instances_per_sequence': 1,
        'config.estimator_kwargs.batch_size': {'$in': [128, 256]},
        'config.dp_optimizer_kwargs.max_grad_norm': {'$in': [0.001, 0.0001]},
        'config.dp_optimizer_kwargs.noise_multiplier': {'$in': [2.0, 4.0]},
        'config.estimator_kwargs.relative_context_length': {'$in': [8]},
        'config.dp_accountant_kwargs.budget_delta': 1e-7,
        'config.estimator_kwargs.lags_seq': {'$in': [None, [
            1, 2, 3, 4, 5, 6, 7, 23, 24, 25, 47, 48, 49, 71, 72, 73,
            95, 96, 97, 119, 120, 121, 143, 144, 145, 167, 168, 169]]},
        'config.tight_privacy_loss': True
    }
)

/nfs/staff-ssd/schuchaj/miniconda3/envs/timeseries_dp/lib/python3.11/site-packag
es/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

In [19]:
columns = df.columns[1:]

df = df.rename(columns={
    c: c.split('.')[-1]
    for c in columns
})

In [20]:
df

,_id,budget_epsilon,max_grad_norm,noise_multiplier,batch_size,relative_context_length,estimator_name,size,seed,MAPE,mean_wQuantileLoss
0,1931,0.5,0.0001,2.0,128,8,SimpleFeedForwardEstimator,1,0,0.598530,0.224734
1,1932,1.0,0.0001,2.0,128,8,SimpleFeedForwardEstimator,1,0,0.487749,0.198488
2,1933,2.0,0.0001,2.0,128,8,SimpleFeedForwardEstimator,1,0,0.443464,0.191161
3,1934,4.0,0.0001,2.0,128,8,SimpleFeedForwardEstimator,1,0,0.439262,0.192664
4,1935,8.0,0.0001,2.0,128,8,SimpleFeedForwardEstimator,1,0,0.443464,0.191161
...,...,...,...,...,...,...,...,...,...,...,...
1595,8316,0.5,0.0001,4.0,256,8,ITransformerEstimator,8,4,1.299420,0.454630
1596,8317,1.0,0.0001,4.0,256,8,ITransformerEstimator,8,4,1.345944,0.348373
1597,8318,2.0,0.0001,4.0,256,8,ITransformerEstimator,8,4,0.813989,0.267429
1598,8319,4.0,0.0001,4.0,256,8,ITransformerEstimator,8,4,0.483811,0.216987


In [21]:
def create_table(df: pd.DataFrame,
                 batch_size: int,
                 max_grad_norm: float,
                 noise_multiplier: float,
                 budget_epsilon: float,
                 errors: bool = False
) -> pd.DataFrame:

    df = df.copy()

    df = df[
        (df['batch_size'] == batch_size)
        & (df['max_grad_norm'] == max_grad_norm)
        & (df['noise_multiplier'] == noise_multiplier)
        & (df['budget_epsilon'] == budget_epsilon)
    ]

    print(len(df))

    # Average over random seeds
    if not errors:
        df = df.groupby(
            ['max_grad_norm', 'noise_multiplier', 'batch_size', 'estimator_name', 'budget_epsilon', 'size']
        ).mean().reset_index()
    else:
        df = df.groupby(
            ['max_grad_norm', 'noise_multiplier', 'batch_size', 'estimator_name', 'budget_epsilon', 'size']
        ).std().reset_index()

    print(df.columns)

    df = df.drop(columns=[
        '_id',
        'seed', 'relative_context_length',
        'max_grad_norm', 'noise_multiplier', 'batch_size', '_id', 'budget_epsilon'])

    df = df.sort_values(by=['estimator_name', 'size'])

    return df

### Batch size 256, Noise Multiplier 4.0

In [22]:
df_mean = create_table(df, 256, 0.0001, 4.0, 4.0)
df_mean

80
Index(['max_grad_norm', 'noise_multiplier', 'batch_size', 'estimator_name',
       'budget_epsilon', 'size', '_id', 'relative_context_length', 'seed',
       'MAPE', 'mean_wQuantileLoss'],
      dtype='object')


,estimator_name,size,MAPE,mean_wQuantileLoss
0,DLinearEstimator,1,0.431685,0.189311
1,DLinearEstimator,2,0.431090,0.188753
2,DLinearEstimator,4,0.440061,0.191662
3,DLinearEstimator,8,0.492122,0.208141
4,DeepAREstimator,1,0.228948,0.141968
5,DeepAREstimator,2,0.235972,0.142739
6,DeepAREstimator,4,0.236807,0.145432
7,DeepAREstimator,8,0.282934,0.166430
8,ITransformerEstimator,1,0.388071,0.188426
9,ITransformerEstimator,2,0.388265,0.188454


In [23]:
df_std = create_table(df, 256, 0.0001, 4.0, 4.0, errors=True)
df_std

80
Index(['max_grad_norm', 'noise_multiplier', 'batch_size', 'estimator_name',
       'budget_epsilon', 'size', '_id', 'relative_context_length', 'seed',
       'MAPE', 'mean_wQuantileLoss'],
      dtype='object')


,estimator_name,size,MAPE,mean_wQuantileLoss
0,DLinearEstimator,1,0.002219,0.002379
1,DLinearEstimator,2,0.002539,0.002982
2,DLinearEstimator,4,0.007687,0.001094
3,DLinearEstimator,8,0.015172,0.004495
4,DeepAREstimator,1,0.010039,0.002534
5,DeepAREstimator,2,0.008321,0.001036
6,DeepAREstimator,4,0.010636,0.001323
7,DeepAREstimator,8,0.010245,0.004080
8,ITransformerEstimator,1,0.007052,0.003659
9,ITransformerEstimator,2,0.006511,0.003619


In [24]:
i = 0
names = ['DLinear', 'DeepAR', 'iTransf.', 'SimpleFF']

acc = f'{names[0]} &'

for i, (mean, std) in enumerate(zip(df_mean['mean_wQuantileLoss'], df_std['mean_wQuantileLoss'])):

    if (i > 0) and ((i % 4)  == 0):
        acc += ' \\\\'
        print(acc)
        acc = f'{names[i // 4]} &'
    acc += ' $' + f'{mean:.3f}' + '$ '
    acc += '\\tiny{$\\pm ' + f'{std:.3f}' + '$} &'

acc += ' \\\\ '
print(acc)

DLinear & $0.189$ \tiny{$\pm 0.002$} & $0.189$ \tiny{$\pm 0.003$} & $0.192$ \tiny{$\pm 0.001$} & $0.208$ \tiny{$\pm 0.004$} & \\
DeepAR & $0.142$ \tiny{$\pm 0.003$} & $0.143$ \tiny{$\pm 0.001$} & $0.145$ \tiny{$\pm 0.001$} & $0.166$ \tiny{$\pm 0.004$} & \\
iTransf. & $0.188$ \tiny{$\pm 0.004$} & $0.188$ \tiny{$\pm 0.004$} & $0.193$ \tiny{$\pm 0.002$} & $0.217$ \tiny{$\pm 0.005$} & \\
SimpleFF & $0.193$ \tiny{$\pm 0.003$} & $0.194$ \tiny{$\pm 0.003$} & $0.194$ \tiny{$\pm 0.003$} & $0.211$ \tiny{$\pm 0.001$} & \\ 
